In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Read your data
df = pd.read_csv('data2.csv')

# Clean and preprocess
df['lemmatized-comment'].replace('', np.nan, inplace=True)
df.dropna(subset=['lemmatized-comment'], inplace=True)

# Split the data
train_text, temp_text, train_labels, temp_labels = train_test_split(df['lemmatized-comment'], df['numerical-label'], 
                                                                    random_state=2018, 
                                                                    test_size=0.25, 
                                                                    stratify=df['numerical-label'])
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [2]:
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import tqdm, trange

# Tokenization
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_text.tolist(), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_text.tolist(), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_text.tolist(), truncation=True, padding=True, max_length=256)

# Convert inputs to PyTorch tensors
train_inputs = torch.tensor(train_encodings['input_ids'])
validation_inputs = torch.tensor(val_encodings['input_ids'])
train_labels = torch.tensor(train_labels.tolist())
validation_labels = torch.tensor(val_labels.tolist())
train_masks = torch.tensor(train_encodings['attention_mask'])
validation_masks = torch.tensor(val_encodings['attention_mask'])
test_inputs = torch.tensor(test_encodings['input_ids'])
test_labels = torch.tensor(test_labels.tolist())
test_masks = torch.tensor(test_encodings['attention_mask'])

# Create DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=16)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=16)

In [3]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the previously saved model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 16)
model.load_state_dict(torch.load("bert_model2.pt"))  # Load the state dict from the saved model
model.to(device)

# Optimization setup
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*3) # assuming 3 epochs


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

for _ in trange(epochs, desc="Epoch"):  
    # Training
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        if step % 250 == 0:
            print("Batch: {0}/{1}, Loss: {2:.2f}".format(step, len(train_dataloader), loss.item()))

    avg_train_loss = total_loss / len(train_dataloader)            
    train_loss_set.append(avg_train_loss)
    print("Average training loss: {0:.2f}".format(avg_train_loss))

    # Validation
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    torch.save(model.state_dict(), f'checkpoint3_{_}.pt') # Save the model checkpoint

# Save the final model
torch.save(model.state_dict(), 'bert_model3.pt')

def evaluate_test_set(test_dataloader):
    # Put model in evaluation mode to evaluate loss on the test set
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in test_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Test Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

# Call the function to evaluate the test set
evaluate_test_set(test_dataloader)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Furkan\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Batch: 0/83949, Loss: 2.01
Batch: 250/83949, Loss: 1.64
Batch: 500/83949, Loss: 2.06
Batch: 750/83949, Loss: 1.95
Batch: 1000/83949, Loss: 2.32
Batch: 1250/83949, Loss: 1.46
Batch: 1500/83949, Loss: 1.71
Batch: 1750/83949, Loss: 2.01
Batch: 2000/83949, Loss: 1.26
Batch: 2250/83949, Loss: 1.82
Batch: 2500/83949, Loss: 1.84
Batch: 2750/83949, Loss: 1.70
Batch: 3000/83949, Loss: 2.05
Batch: 3250/83949, Loss: 1.64
Batch: 3500/83949, Loss: 1.71
Batch: 3750/83949, Loss: 1.64
Batch: 4000/83949, Loss: 1.35
Batch: 4250/83949, Loss: 1.48
Batch: 4500/83949, Loss: 1.73
Batch: 4750/83949, Loss: 1.46
Batch: 5000/83949, Loss: 1.80
Batch: 5250/83949, Loss: 1.72
Batch: 5500/83949, Loss: 1.92
Batch: 5750/83949, Loss: 1.68
Batch: 6000/83949, Loss: 1.91
Batch: 6250/83949, Loss: 2.08
Batch: 6500/83949, Loss: 1.91
Batch: 6750/83949, Loss: 2.74
Batch: 7000/83949, Loss: 1.99
Batch: 7250/83949, Loss: 1.56
Batch: 7500/83949, Loss: 1.66
Batch: 7750/83949, Loss: 1.63
Batch: 8000/83949, Loss: 2.15
Batch: 8250/8394

Epoch:  25%|██▌       | 1/4 [11:39:15<34:57:45, 41955.02s/it]

Batch: 0/83949, Loss: 2.30
Batch: 250/83949, Loss: 2.07
Batch: 500/83949, Loss: 1.95
Batch: 750/83949, Loss: 1.96
Batch: 1000/83949, Loss: 2.17
Batch: 1250/83949, Loss: 1.08
Batch: 1500/83949, Loss: 2.14
Batch: 1750/83949, Loss: 1.71
Batch: 2000/83949, Loss: 1.69
Batch: 2250/83949, Loss: 1.30
Batch: 2500/83949, Loss: 1.71
Batch: 2750/83949, Loss: 2.18
Batch: 3000/83949, Loss: 1.36
Batch: 3250/83949, Loss: 1.90
Batch: 3500/83949, Loss: 1.52
Batch: 3750/83949, Loss: 1.48
Batch: 4000/83949, Loss: 1.70
Batch: 4250/83949, Loss: 1.87
Batch: 4500/83949, Loss: 1.09
Batch: 4750/83949, Loss: 1.99
Batch: 5000/83949, Loss: 1.60
Batch: 5250/83949, Loss: 2.47
Batch: 5500/83949, Loss: 1.52
Batch: 5750/83949, Loss: 1.75
Batch: 6000/83949, Loss: 1.73
Batch: 6250/83949, Loss: 1.76
Batch: 6500/83949, Loss: 1.21
Batch: 6750/83949, Loss: 1.33
Batch: 7000/83949, Loss: 1.97
Batch: 7250/83949, Loss: 1.75
Batch: 7500/83949, Loss: 1.91
Batch: 7750/83949, Loss: 1.49
Batch: 8000/83949, Loss: 1.67
Batch: 8250/8394

Epoch:  50%|█████     | 2/4 [23:16:53<23:16:36, 41898.42s/it]

Batch: 0/83949, Loss: 1.63
Batch: 250/83949, Loss: 1.49
Batch: 500/83949, Loss: 2.02
Batch: 750/83949, Loss: 1.23
Batch: 1000/83949, Loss: 1.66
Batch: 1250/83949, Loss: 1.49
Batch: 1500/83949, Loss: 1.49
Batch: 1750/83949, Loss: 2.04
Batch: 2000/83949, Loss: 1.74
Batch: 2250/83949, Loss: 1.71
Batch: 2500/83949, Loss: 1.68
Batch: 2750/83949, Loss: 1.55
Batch: 3000/83949, Loss: 2.09
Batch: 3250/83949, Loss: 1.63
Batch: 3500/83949, Loss: 1.53
Batch: 3750/83949, Loss: 1.11
Batch: 4000/83949, Loss: 1.59
Batch: 4250/83949, Loss: 1.31
Batch: 4500/83949, Loss: 1.67
Batch: 4750/83949, Loss: 2.05
Batch: 5000/83949, Loss: 1.98
Batch: 5250/83949, Loss: 1.53
Batch: 5500/83949, Loss: 1.99
Batch: 5750/83949, Loss: 2.37
Batch: 6000/83949, Loss: 1.86
Batch: 6250/83949, Loss: 2.07
Batch: 6500/83949, Loss: 1.63
Batch: 6750/83949, Loss: 1.57
Batch: 7000/83949, Loss: 2.68
Batch: 7250/83949, Loss: 1.74
Batch: 7500/83949, Loss: 1.70
Batch: 7750/83949, Loss: 1.17
Batch: 8000/83949, Loss: 1.87
Batch: 8250/8394

Epoch:  75%|███████▌  | 3/4 [34:51:48<11:36:45, 41805.49s/it]

Batch: 0/83949, Loss: 1.44


Epoch:  75%|███████▌  | 3/4 [34:52:49<11:37:36, 41856.42s/it]


KeyboardInterrupt: 